# 循环神经网络（Recurrent Neural Network，RNN）

## 1.概述
RNN 是一种深度学习模型，它能够处理序列数据，如文本、音频、视频等。它可以记住之前的输入，并利用这些信息对当前输入做出更好的预测。RNNs 通常由多个隐藏层组成，每个隐藏层中都有多个神经元，每个神经元与前一时刻的输出和当前时刻的输入相连。RNNs 能够捕捉时间序列数据中的长期依赖关系，并对未来数据进行预测。

## 2. 词嵌入层
词嵌入层的作用就是将文本转换为向量。词嵌入层首先会根据输入的词的数量构建一个词向量矩阵，每一行代表一个词，每一列代表一个词向量（维数自定）。

在 PyTorch 中，我们可以使用 nn.Embedding 词嵌入层来实现输入词的向量化。

具体步骤：
1. Tokenize 输入的文本，构建词与词索引的映射关系。
2. 使用 nn.Embedding 构建词嵌入矩阵，词索引对应的向量即为该词对应的数值化后的向量表示。

注意：词嵌入层中的向量表示是可学习的，并不是固定不变的。

In [1]:
import torch
import torch.nn as nn
import jieba

text = '北京冬奥的进度条已经过半，不少外国运动员在完成自己的比赛后踏上归途。'
# 1. Tokenize
words = jieba.lcut(text)
print(words)
# 2. 构建词表
index_to_word = {}
word_to_index = {}
unique_words = list(set(words))  # 去重

for i, word in enumerate(unique_words):
    index_to_word[i] = word
    word_to_index[word] = i

# 3. 构建词嵌入层
'''
num_embeddings: 词表大小
embedding_dim: 词向量维度
'''
embed = nn.Embedding(num_embeddings=len(index_to_word), embedding_dim=4)
print('-' * 50)
# 4. 文本转换为词向量
for word in words:
    index = word_to_index[word]
    vector = embed(torch.tensor(index))
    print('%3s -> %s' % (word, vector))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Ignorant\AppData\Local\Temp\jieba.cache
Loading model cost 0.576 seconds.
Prefix dict has been built successfully.


['北京', '冬奥', '的', '进度条', '已经', '过半', '，', '不少', '外国', '运动员', '在', '完成', '自己', '的', '比赛', '后', '踏上', '归途', '。']
--------------------------------------------------
 北京 -> tensor([-0.2493, -2.0450,  1.3385, -0.1441], grad_fn=<EmbeddingBackward0>)
 冬奥 -> tensor([-0.3383,  0.8249,  0.0257, -0.9948], grad_fn=<EmbeddingBackward0>)
  的 -> tensor([-0.9397, -0.0259,  0.5528,  0.2483], grad_fn=<EmbeddingBackward0>)
进度条 -> tensor([-0.5763, -0.5205,  1.0997,  0.6395], grad_fn=<EmbeddingBackward0>)
 已经 -> tensor([ 1.5045,  2.1313, -0.2308,  1.0335], grad_fn=<EmbeddingBackward0>)
 过半 -> tensor([-2.4331, -2.2740,  0.9175,  0.0867], grad_fn=<EmbeddingBackward0>)
  ， -> tensor([ 0.9867, -1.3687, -0.3180, -1.6667], grad_fn=<EmbeddingBackward0>)
 不少 -> tensor([-0.1120,  1.5962,  0.0292,  0.3414], grad_fn=<EmbeddingBackward0>)
 外国 -> tensor([-0.5455, -0.0541, -0.2291, -0.2168], grad_fn=<EmbeddingBackward0>)
运动员 -> tensor([ 1.1138e+00, -1.5857e+00,  1.1528e+00, -1.4141e-03],
       grad_fn=<EmbeddingBackwar

## 3.循环网络层

### 3.1 RNN 网络原理
网络结构：

<img src="images/RNN.png">

其中 h 表示隐藏状态，x 表示输入，y 表示输出。每次输入包含两个值：上一个时间步的隐藏状态和当前时间步的输入。输出当前时间步的隐藏状态，并作为下一个时间步的输入。

实际上上图所描述的过程只用了一个神经元。

RNN 网络可以有多个神经元：

<img src="images/RNN_multi.png">

依次将 "你爱我" 三个字分别送入到每个神经元进行计算，假设词嵌入时，"你爱我" 的维度为 128，经过循环网络之后，"你爱我" 三个字的词向量维度就会变成 4。

每个神经元内部的计算公式：$$ h_t = tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh}) $$

其中 $W_{ih}$ 和 $W_{hh}$ 分别是输入和隐藏状态的权重矩阵，$b_{ih}$ 和 $b_{hh}$ 分别是输入和隐藏状态的偏置项。

In [2]:
# RNN 送入单个数据
rnn = nn.RNN(input_size=128, hidden_size=256)

inputs = torch.randn(1, 1, 128)     # 输入形状为 (seq_len, batch, input_size)
hn = torch.randn(1, 1, 256)

output, hn = rnn(inputs, hn)
print(output.shape)
print(hn.shape)

torch.Size([1, 1, 256])
torch.Size([1, 1, 256])


In [3]:
# RNN 送入批量数据
rnn = nn.RNN(input_size=128, hidden_size=256)

inputs = torch.randn(1, 32, 128)
hn = torch.randn(1, 32, 256)

output, hn = rnn(inputs, hn)
print(output.shape)
print(hn.shape)

torch.Size([1, 32, 256])
torch.Size([1, 32, 256])
